# VPP Qualification 

Determine if we can bid for FCR-D

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = "../data/raw"
HEATING_MONTHS = [
    1,
    2,
    3,
    4,
    10,
    11,
    12,
]  # Heating season months (Oct-Apr and filter out summer months)

df = pd.read_csv(os.path.join(DATA_DIR, "fcr_d_prices_2024.csv"), delimiter=";")

df["HourUTC"] = pd.to_datetime(df["HourUTC"])
df["Month"] = df["HourUTC"].dt.month
df["PriceTotalEUR"] = df["PriceTotalEUR"].str.replace(",", ".").astype(float)
df["PriceTotalEUR"] = df["PriceTotalEUR"] 

df.head()

,HourUTC,HourDK,PriceArea,ProductName,AuctionType,PurchasedVolumeLocal,PurchasedVolumeTotal,PriceTotalEUR,Month
0,2024-12-31 22:00:00,2024-12-31 23:00:00,SE2,FCR-N,D-1 late,"25,900000","35,000000",6.97,12
1,2024-12-31 22:00:00,2024-12-31 23:00:00,SE2,FCR-D upp,D-1 late,"41,100000","150,300000",0.98,12
2,2024-12-31 22:00:00,2024-12-31 23:00:00,SE2,FCR-D ned,D-1 late,"70,900000","201,500000",0.98,12
3,2024-12-31 22:00:00,2024-12-31 23:00:00,SE3,FCR-N,D-1 late,"2,000000","35,000000",6.97,12
4,2024-12-31 22:00:00,2024-12-31 23:00:00,SE3,FCR-D upp,D-1 late,"62,400000","150,300000",0.98,12


In [2]:
# Filter for "AcutionType = Total" and ProductName = "FCR-D ned" or "FCR-D upp"
df = df[df["AuctionType"] == "Total"]
df = df[df["ProductName"].isin(["FCR-D ned", "FCR-D upp"])]
df = df[df["PriceArea"].isin(["DK1", "DK2"])]
df.head()

,HourUTC,HourDK,PriceArea,ProductName,AuctionType,PurchasedVolumeLocal,PurchasedVolumeTotal,PriceTotalEUR,Month
41,2024-12-31 22:00:00,2024-12-31 23:00:00,DK2,FCR-D upp,Total,"34,600000","605,600000",5.130026,12
42,2024-12-31 22:00:00,2024-12-31 23:00:00,DK2,FCR-D ned,Total,"67,300000","554,000000",1.819892,12
48,2024-12-31 21:00:00,2024-12-31 22:00:00,DK2,FCR-D ned,Total,"69,500000","549,500000",1.701865,12
49,2024-12-31 21:00:00,2024-12-31 22:00:00,DK2,FCR-D upp,Total,"34,600000","604,000000",5.155050,12
130,2024-12-31 20:00:00,2024-12-31 21:00:00,DK2,FCR-D upp,Total,"31,000000","582,800000",5.303363,12


In [3]:
# Get the average price for each product during the heating season
df_heating_season = df[df["Month"].isin(HEATING_MONTHS)]
avg_price_fcr_d_ned = df_heating_season[df_heating_season["ProductName"] == "FCR-D ned"]["PriceTotalEUR"].mean()
avg_price_fcr_d_upp = df_heating_season[df_heating_season["ProductName"] == "FCR-D upp"]["PriceTotalEUR"].mean()
print(f"Average price for FCR-D ned: {avg_price_fcr_d_ned:.2f} EUR/MW")
print(f"Average price for FCR-D upp: {avg_price_fcr_d_upp:.2f} EUR/MW")


Average price for FCR-D ned: 35.84 EUR/MW
Average price for FCR-D upp: 11.63 EUR/MW


In [4]:
# Revenue calculation for 5 kW heaters (symmetric FCR-D bidding)

# Key assumptions
HEATER_CAPACITY_KW = 5              # kW per household
HOUSEHOLDS_PER_MW = 1000 / HEATER_CAPACITY_KW  # households needed for 1 MW
BID_ACCEPTANCE_RATE = 0.50          # Conservative market clearing assumption

# Calculate heating season hours from actual data
heating_hours = df_heating_season["HourUTC"].nunique()
print(f"Heating season hours (from data): {heating_hours}")

# Combined price for symmetric bidding (providing both up and down regulation)
combined_price = avg_price_fcr_d_upp + avg_price_fcr_d_ned
print(f"\nCombined price (up + down): {combined_price:.2f} EUR/MW/h")

# Annual revenue calculations for 1 MW portfolio
print(f"\n--- Revenue for 1 MW portfolio ({int(HOUSEHOLDS_PER_MW)} households with {HEATER_CAPACITY_KW} kW heaters) ---")

# Scenario 1: Symmetric bidding (both up and down)
revenue_symmetric = heating_hours * combined_price * BID_ACCEPTANCE_RATE
print(f"\nSymmetric bidding (FCR-D up + down):")
print(f"  Annual revenue (1 MW): {revenue_symmetric:,.0f} EUR")
print(f"  Per household: {revenue_symmetric / HOUSEHOLDS_PER_MW:,.0f} EUR")

# Scenario 2: FCR-D up only (heaters reduce consumption)
revenue_up_only = heating_hours * avg_price_fcr_d_upp * BID_ACCEPTANCE_RATE
print(f"\nFCR-D up only:")
print(f"  Annual revenue (1 MW): {revenue_up_only:,.0f} EUR")
print(f"  Per household: {revenue_up_only / HOUSEHOLDS_PER_MW:,.0f} EUR")

# Scenario 3: FCR-D down only (heaters increase consumption)
revenue_down_only = heating_hours * avg_price_fcr_d_ned * BID_ACCEPTANCE_RATE
print(f"\nFCR-D down only:")
print(f"  Annual revenue (1 MW): {revenue_down_only:,.0f} EUR")
print(f"  Per household: {revenue_down_only / HOUSEHOLDS_PER_MW:,.0f} EUR")

print(f"\n--- Assumptions ---")
print(f"Heater capacity: {HEATER_CAPACITY_KW} kW")
print(f"Bid acceptance rate: {BID_ACCEPTANCE_RATE*100:.0f}%")
print(f"Heating season: {HEATING_MONTHS}")

Heating season hours (from data): 5112

Combined price (up + down): 47.47 EUR/MW/h

--- Revenue for 1 MW portfolio (200 households with 5 kW heaters) ---

Symmetric bidding (FCR-D up + down):
  Annual revenue (1 MW): 121,330 EUR
  Per household: 607 EUR

FCR-D up only:
  Annual revenue (1 MW): 29,725 EUR
  Per household: 149 EUR

FCR-D down only:
  Annual revenue (1 MW): 91,604 EUR
  Per household: 458 EUR

--- Assumptions ---
Heater capacity: 5 kW
Bid acceptance rate: 50%
Heating season: [1, 2, 3, 4, 10, 11, 12]
